## Run Gordo from only a config file

This is a higher level example of how gordo works.

Train a model from a config file.

---
** Some slight difference in how it _actually_ works, in that we normally use some resources from `gordo-infrastructure` for parsing the config that we don't have access to here. Yet.

---

In [1]:
import tempfile
import yaml
from pprint import pprint
from dateutil.parser import isoparse

from gordo import serializer
from gordo.builder import local_build
from gordo_dataset.data_provider.providers import DataLakeProvider

## Define some config file:

In [2]:
config = \
"""
machines:
  - dataset:
      tags:
        - SOME-TAG1
        - SOME-TAG2
      target_tag_list:
        - SOME-TAG3
        - SOME-TAG4
      train_end_date: '2019-03-01T00:00:00+00:00'
      train_start_date: '2019-01-01T00:00:00+00:00'
      asset: asgb
      data_provider:
        type: RandomDataProvider
    metadata:
      information: Some sweet information about the model
    model:
      gordo.machine.model.anomaly.diff.DiffBasedAnomalyDetector:
        base_estimator:
          sklearn.pipeline.Pipeline:
            steps:
            - sklearn.decomposition.PCA
            - sklearn.multioutput.MultiOutputRegressor:
                estimator: sklearn.linear_model.LinearRegression
    name: crazy-sweet-name
"""

### Build model from data and model configs

`gordo.builder.local_build` is a generator for building each machine in your string config, much in the same actual deployment on k8s works. 

Here we'll simply cast it as a list and get the first element, which is a `Tuple[BaseEstimator, dict]` of your model and metadata

In [3]:
pipe, metadata = next(local_build(config))

---

### The trained model/pipeline:

In [4]:
pipe

DiffBasedAnomalyDetector(base_estimator=Pipeline(memory=None,
                                                 steps=[('step_0',
                                                         PCA(copy=True,
                                                             iterated_power='auto',
                                                             n_components=None,
                                                             random_state=None,
                                                             svd_solver='auto',
                                                             tol=0.0,
                                                             whiten=False)),
                                                        ('step_1',
                                                         MultiOutputRegressor(estimator=LinearRegression(copy_X=True,
                                                                                                         fit_intercept=True,
                  

### Metadata from the model and build process

In [5]:
print(yaml.dump(metadata))

dataset:
  asset: asgb
  data_provider:
    type: RandomDataProvider
  filter: ''
  resolution: 10T
  row_filter_buffer_size: 0
  tag_list:
  - !!python/object/new:gordo.machine.dataset.sensor_tag.SensorTag
    - SOME-TAG1
    - asgb
  - !!python/object/new:gordo.machine.dataset.sensor_tag.SensorTag
    - SOME-TAG2
    - asgb
  target_tag_list:
  - !!python/object/new:gordo.machine.dataset.sensor_tag.SensorTag
    - SOME-TAG3
    - asgb
  - !!python/object/new:gordo.machine.dataset.sensor_tag.SensorTag
    - SOME-TAG4
    - asgb
  train_end_date: 2019-03-01 00:00:00+00:00
  train_start_date: 2019-01-01 00:00:00+00:00
model:
  base_estimator: "Pipeline(memory=None,\n         steps=[('step_0',\n           \
    \      PCA(copy=True, iterated_power='auto', n_components=None,\n            \
    \         random_state=None, svd_solver='auto', tol=0.0,\n                   \
    \  whiten=False)),\n                ('step_1',\n                 MultiOutputRegressor(estimator=LinearRegression(co